In [1]:
import requests
import time 
import pandas as pd
 
from bs4 import BeautifulSoup

In [15]:

url = "https://www.ss.com/en/agriculture/agricultural-machinery/tractors/"
url

'https://www.ss.com/en/agriculture/agricultural-machinery/tractors/'

In [3]:
req = requests.get(url) 
req.status_code

200

In [4]:
soup = BeautifulSoup(req.text, 'lxml') 
soup.title

<title>SS.COM Agricultural machinery - Tractors, Prices - Advertisements</title>

In [5]:
def getColList(soup):
    column_list = ["description","url"] 
    headline = soup.find("tr", {"id":"head_line"})
    headtds = headline.find_all("td")
    headcolumns = [el.text for el in headtds[1:]] 
    column_list += headcolumns
    return column_list

In [6]:
column_names = getColList(soup)
column_names

['description', 'url', 'Brand', 'Year', 'Price']

In [7]:
def getRowList(soup):
    trows = soup.find_all('tr')
    aprows = [row for row in trows if row.get('id',"").startswith("tr_") and not row.get('id',"").startswith("tr_bnr") ]
    return aprows

In [8]:
def getRow(row, colist=column_names):
    row_tds = row.find_all('td')
    rowDict = {}
    if len(row_tds) < 3: # a little sanity check
        print("Hmm bad row")
        return rowDict
    
    rowDict[colist[0]] = row_tds[2].text # so the big assumption is that we always get description in 3rd column
    rowDict[colist[1]] = "https://ss.com" + row_tds[1].find('a').get('href')
    for td,key in zip(row_tds[3:],colist[2:]): 
        rowDict[key] = td.text
    return rowDict

In [9]:
def getRows(rowlist,colist=column_names):
    return [getRow(row, colist=colist) for row in rowlist]

In [10]:
def getDFfromURL(url):
    
    req = requests.get(url)
    if req.status_code != 200:
        print("Request Fail with", req.status_code)
        return None 
    soup = BeautifulSoup(req.text, 'lxml')
    column_names = getColList(soup)
    rowlist = getRowList(soup)
    rows = getRows(rowlist,colist=column_names)
    return pd.DataFrame(rows, columns=column_names)

In [11]:
tractors = "https://www.ss.com/en/agriculture/agricultural-machinery/tractors/"
idf = getDFfromURL(tractors)
idf.head()

,description,url,Brand,Year,Price
0,Cena norādīta ar Pvn. Pārdodu T150-K labā darb...,https://ss.com/msg/en/agriculture/agricultural...,T150-KT150-K,1989,"13,200 €"
1,"New Holland T7, 250 zirgi, autocomand, priekse...",https://ss.com/msg/en/agriculture/agricultural...,New HollandT7 250,2013,"64,000 €"
2,Pārdodu T40 ar tehnisko un dokumentiem. Darba ...,https://ss.com/msg/en/agriculture/agricultural...,T40Am,1991,"3,400 €"
3,"Nopirktu mtz 82Daugavpils and reg., Līksnas pag.",https://ss.com/msg/en/agriculture/agricultural...,BelarusMtz 82,-,buy
4,"Pārdod darbaspējīgu traktoru T25, ar dokumenti...",https://ss.com/msg/en/agriculture/agricultural...,T25T25,-,"3,350 €"


In [12]:
idf.to_csv("traktoru_pardosana.csv", encoding="utf-8")  # ... and here I get error in opening the created .csv file and as a result error for the further step

In [13]:
traktori = pd.read_csv('C:\Users\zirav\Desktop\Accenture\Day20_WebScraping\traktoru_pardosana.csv', encoding="utf-8")
traktori

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1904089634.py, line 1)